In [1]:
from collections import OrderedDict
import re
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from tqdm.notebook import tqdm

from eval import * 
from plot_utils import *
from models import *
from samplers import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set_theme('notebook', 'darkgrid')
palette = sns.color_palette('colorblind')

In [2]:
run_dir = "./models"

df = read_run_dir(run_dir)
task = "linear_regression"
#task = "sparse_linear_regression"
#task = "decision_tree"
#task = "relu_2nn_regression"

run_id = "pretrained"  # if you train more models, replace with the run_id from the table above

run_path = os.path.join(run_dir, task, run_id)

_, conf = get_model_from_run(run_path, only_conf=True)

In [3]:
#### Generate skewed points

In [4]:
result = build_evals(conf)
skewed_sampler = GaussianSampler(n_dims=20, scale=result['skewed']['data_sampler_kwargs']['scale'])

In [6]:
from samplers import get_data_sampler
from tasks import get_task_sampler

model, conf = get_model_from_run(run_path)

n_dims = conf.model.n_dims
batch_size = conf.training.batch_size

task_sampler = get_task_sampler(
    conf.training.task,
    n_dims,
    batch_size,
    **conf.training.task_kwargs
)

In [14]:
task = task_sampler()

xs = skewed_sampler.sample_xs(b_size=batch_size, n_points=conf.training.curriculum.points.end)
ys = task.evaluate(xs)

In [15]:
with torch.no_grad():
    pred = model(xs, ys)
metric = task.get_metric()
loss = metric(pred, ys).numpy()

In [17]:
print(["%.3f" % (val/20) for val in loss.mean(axis=0)])

['0.880', '0.912', '0.441', '0.467', '0.259', '0.300', '0.161', '0.113', '0.135', '0.117', '0.113', '0.117', '0.087', '0.085', '0.047', '0.150', '0.124', '0.292', '0.136', '0.219', '0.136', '0.181', '0.216', '0.133', '0.222', '0.171', '0.117', '0.128', '0.197', '0.185', '0.193', '0.246', '0.143', '0.178', '0.238', '0.156', '0.199', '0.222', '0.117', '0.163', '0.150']
